In [2]:
import tensorflow as tf
import sys
sys.path.append('/nas/longleaf/home/siyangj/NiftyNet/')
import niftynet as nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import SimpleITK as sitk
import os
import re
import nibabel as nib
import sklearn as skl
import pandas as pd

data_path = '/proj/NIRAL/users/siyangj/myData/min_cut_outside/'
model = '10112050'

INFO:tensorflow:TensorFlow version 1.9.0
CRITICAL:tensorflow:Optional Python module cv2 not found, please install cv2 and retry if the application fails.
INFO:tensorflow:Available Image Loaders:
['nibabel', 'skimage', 'pillow', 'simpleitk', 'dummy'].


In [15]:
def model_eval(model,data_path,eval_df,test_im):
    model_path = '/proj/NIRAL/users/siyangj/model_{}/'.format(model)
    inf_re = re.compile(r'inference.*')
    im_re = re.compile(r'.*\.nii\.gz')
    inf_dirs = [f for f in os.listdir(model_path) 
            if (os.path.isdir(os.path.join(model_path, f)) 
                and inf_re.match(f) is not None)]
    for d in inf_dirs:
        inf_dir = os.path.join(model_path,d)
        inf_ims = {f[0]:os.path.join(inf_dir,f) for f in os.listdir(inf_dir) 
                if (os.path.isfile(os.path.join(inf_dir, f)) and im_re.match(f) is not None)}
        l = len(eval_df.index)
        eval_df.loc[l,'inf_dir']=inf_dir
        eval_df.loc[l,'model']=model
        eval_df.loc[l,'test_im']=test_im
        for im_num,im in inf_ims.items():
            true_im = os.path.join(data_path,'seg/',im_num+'.nrrd')
            true_arr = sitk.GetArrayFromImage(sitk.ReadImage(true_im))
            #print(true_arr.shape)
            inf_arr = np.swapaxes(nib.load(im).get_fdata().squeeze(),0,2)
            #print(inf_arr.shape)
            c_mat = skl.metrics.confusion_matrix(true_arr.flatten(),inf_arr.flatten())
            p_mat = c_mat.T / c_mat.astype(np.float).sum(axis=1)
            eval_df.loc[l,im_num]=[p_mat.T]
            if im_num == test_im:
                eval_df.loc[l,'test_11'] = p_mat[1,1]
                eval_df.loc[l,'test_12'] = p_mat[2,1]
                eval_df.loc[l,'test_21'] = p_mat[1,2]
                eval_df.loc[l,'test_22'] = p_mat[2,2]
                eval_df.loc[l,'test_11+22'] = eval_df.loc[l,'test_11']+eval_df.loc[l,'test_22']
                eval_df.loc[l,'test_12+21'] = eval_df.loc[l,'test_12']+eval_df.loc[l,'test_21']

In [25]:
eval_df = pd.DataFrame(columns=['model','inf_dir','test_im','test_11','test_12','test_21','test_22',
                                'test_11+22','test_12+21',
                                '1','2','3','4','5','6','7','8'],dtype=object)

for i in range (1,9):
    eval_df[str(i)] = eval_df[str(i)].astype(object)

In [28]:
models = ['10112050',
         '10112055',
         '10112056',
         '10112103']
for m in models:
    model_eval(m,data_path,eval_df,'1')

models = ['10112127','10112130']
data_path = '/proj/NIRAL/users/siyangj/myData/min_fill_outside/'

for m in models:
    model_eval(m,data_path,eval_df,'1')

KeyboardInterrupt: 

In [32]:
eval_df

,model,inf_dir,test_im,test_11,test_12,test_21,test_22,1,2,3,4,5,6,7,8
0,10112050,/proj/NIRAL/users/siyangj/model_10112050/infer...,1,0.918915,0.0673993,0.295044,0.649303,"[[[0.9999979550520439, 0.0, 0.0, 2.04494795607...","[[[0.9928863284474198, 2.4221511310514186e-05,...","[[[0.9999966100890529, 0.0, 0.0, 3.38991094703...","[[[1.0, 0.0, 0.0, 0.0], [0.0, 0.92960586709740...","[[[0.9999399400687686, 0.0, 2.144997543977812e...","[[[0.9999654490926931, 0.0, 0.0, 3.45509073068...","[[[0.9999742775418933, 0.0, 0.0, 2.57224581066...","[[[0.9998266042001417, 0.0, 8.256942850395755e..."
1,10112050,/proj/NIRAL/users/siyangj/model_10112050/infer...,1,0.895175,0.097146,0.224378,0.75351,"[[[1.0, 0.0, 0.0, 0.0], [6.643598615916955e-05...","[[[0.9928434750043319, 0.0, 0.0067689808146998...","[[[0.9999694908014767, 0.0, 1.0169732841118263...","[[[1.0, 0.0, 0.0, 0.0], [3.0097968888732824e-0...","[[[0.9999742600294723, 0.0, 4.289995087955624e...","[[[0.9999861796370773, 0.0, 1.7275453653412938...","[[[0.9999931406778382, 0.0, 0.0, 6.85932216178...","[[[0.9999223847372063, 0.0, 2.807360569134557e..."
2,10112050,/proj/NIRAL/users/siyangj/model_10112050/infer...,1,0.926572,0.0585024,0.308779,0.649881,"[[[0.9999918202081757, 0.0, 0.0, 8.17979182429...","[[[0.9958488056000134, 0.0, 0.0032233241974761...","[[[0.9999762706233707, 0.0, 1.0169732841118263...","[[[1.0, 0.0, 0.0, 0.0], [2.5081640740610688e-0...","[[[0.9999828400196482, 0.0, 8.579990175911249e...","[[[0.9999948173639039, 0.0, 1.7275453653412938...","[[[0.9999948555083786, 0.0, 0.0, 5.14449162133...","[[[0.9999488069543275, 0.0, 1.1559719990554058..."
3,10112055,/proj/NIRAL/users/siyangj/model_10112055/infer...,1,0.824332,0.159098,0.141585,0.805546,"[[[0.9999570560929224, 0.0, 0.0, 4.29439070775...","[[[0.9904772196686125, 2.9811090843709765e-05,...","[[[0.9999711857569502, 1.6949554735197106e-06,...","[[[1.0, 0.0, 0.0, 0.0], [2.006531259248855e-05...","[[[0.9999270700835048, 0.0, 8.579990175911249e...","[[[0.9999758143648853, 0.0, 5.182636096023882e...","[[[0.9999879961862169, 0.0, 0.0, 1.20038137831...","[[[0.9998612833601134, 0.0, 6.275276566300774e..."
4,10112055,/proj/NIRAL/users/siyangj/model_10112055/infer...,1,0.892019,0.0944388,0.250173,0.696974,"[[[1.0, 0.0, 0.0, 0.0], [4.4290657439446366e-0...","[[[0.9923199177213893, 3.7263863554637206e-06,...","[[[0.9999949151335794, 0.0, 0.0, 5.08486642055...","[[[1.0, 0.0, 0.0, 0.0], [5.016328148122138e-06...","[[[0.9999699700343843, 0.0, 0.0, 3.00299656156...","[[[0.9999861796370773, 0.0, 0.0, 1.38203629227...","[[[0.9999948555083786, 0.0, 0.0, 5.14449162133...","[[[0.9999322930686267, 0.0, 1.3211108560633208..."
5,10112055,/proj/NIRAL/users/siyangj/model_10112055/infer...,1,0.861016,0.12516,0.120896,0.821308,"[[[0.9999959101040878, 0.0, 0.0, 4.08989591214...","[[[0.9953215219307153, 3.7263863554637206e-06,...","[[[1.0, 0.0, 0.0, 0.0], [2.337281391149884e-05...","[[[1.0, 0.0, 0.0, 0.0], [1.0032656296244275e-0...","[[[0.9999849850171921, 0.0, 0.0, 1.50149828078...","[[[0.9999879071824426, 0.0, 1.7275453653412938...","[[[0.9999965703389191, 0.0, 0.0, 3.42966108089...","[[[0.9999669722285984, 0.0, 4.954165710237453e..."
6,10112056,/proj/NIRAL/users/siyangj/model_10112056/infer...,1,0.825539,0.124662,0.174211,0.737446,"[[[0.9999979550520439, 0.0, 0.0, 2.04494795607...","[[[0.9902853107713061, 7.452772710927441e-06, ...","[[[1.0, 0.0, 0.0, 0.0], [3.505922086724826e-05...","[[[1.0, 0.0, 0.0, 0.0], [5.016328148122138e-06...","[[[0.9999656800392963, 0.0, 1.0724987719889061...","[[[0.9999827245463466, 0.0, 0.0, 1.72754536534...","[[[0.9999897110167574, 0.0, 0.0, 1.02889832426...","[[[0.9999256875143464, 1.651388570079151e-06, ..."
7,10112056,/proj/NIRAL/users/siyangj/model_10112056/infer...,1,0.793855,0.189398,0.0904256,0.849576,"[[[1.0, 0.0, 0.0, 0.0], [0.0001162629757785467...","[[[0.9934490127870947, 3.7263863554637206e-06,...","[[[1.0, 0.0, 0.0, 0.0], [4.0902424345122974e-0...","[[[1.0, 0.0, 0.0, 0.0], [1.5048984444366412e-0...","[[[0.999976405

In [36]:
eval_df.loc[5,'1']

[array([[9.99995910e-01, 0.00000000e+00, 0.00000000e+00, 4.08989591e-06],
        [3.32179931e-05, 8.61015917e-01, 1.25159862e-01, 1.37910035e-02],
        [1.21725242e-03, 1.20895650e-01, 8.21308120e-01, 5.65789780e-02],
        [9.59473800e-02, 1.55737627e-02, 7.58802665e-02, 8.12598591e-01]])]

In [67]:
eval_df.to_csv('/proj/NIRAL/users/siyangj/inference_eval.csv',index=None)

In [69]:
eval_df = pd.read_csv('/proj/NIRAL/users/siyangj/inference_eval.csv')

In [70]:
eval_df

,model,inf_dir,test_im,test_11,test_12,test_21,test_22,1,2,3,4,5,6,7,8,test_11+22,test_12+21
0,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference_17000_10112050,1,0.918915,0.067399,0.295044,0.649303,"[array([[9.99997955e-01, 0.00000000e+00, 0.00000000e+00, 2.04494796e-06],\n [6.64359862e-0...","[array([[9.92886328e-01, 2.42215113e-05, 2.34389702e-03, 4.74555302e-03],\n [1.10490523e-0...","[array([[9.99996610e-01, 0.00000000e+00, 0.00000000e+00, 3.38991095e-06],\n [0.00000000e+0...","[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n [0.00000000e+0...","[array([[9.99939940e-01, 0.00000000e+00, 2.14499754e-06, 5.79149337e-05],\n [0.00000000e+0...","[array([[9.99965449e-01, 0.00000000e+00, 0.00000000e+00, 3.45509073e-05],\n [0.00000000e+0...","[array([[9.99974278e-01, 0.00000000e+00, 0.00000000e+00, 2.57224581e-05],\n [0.00000000e+0...","[array([[9.99826604e-01, 0.00000000e+00, 8.25694285e-06, 1.65138857e-04],\n [0.00000000e+0...",1.568218,0.362443
1,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference,1,0.895175,0.097146,0.224378,0.753510,"[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n [6.64359862e-0...","[array([[9.92843475e-01, 0.00000000e+00, 6.76898081e-03, 3.87544181e-04],\n [1.21539575e-0...","[array([[9.99969491e-01, 0.00000000e+00, 1.01697328e-05, 2.03394657e-05],\n [1.98668918e-0...","[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n [3.00979689e-0...","[array([[9.99974260e-01, 0.00000000e+00, 4.28999509e-06, 2.14499754e-05],\n [0.00000000e+0...","[array([[9.99986180e-01, 0.00000000e+00, 1.72754537e-06, 1.20928176e-05],\n [0.00000000e+0...","[array([[9.99993141e-01, 0.00000000e+00, 0.00000000e+00, 6.85932216e-06],\n [7.04528001e-0...","[array([[9.99922385e-01, 0.00000000e+00, 2.80736057e-05, 4.95416571e-05],\n [0.00000000e+0...",1.648685,0.321524
2,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference_13200_10112050,1,0.926572,0.058502,0.308779,0.649881,"[array([[9.99991820e-01, 0.00000000e+00, 0.00000000e+00, 8.17979182e-06],\n [1.10726644e-0...","[array([[9.95848806e-01, 0.00000000e+00, 3.22332420e-03, 9.27870203e-04],\n [7.18188397e-0...","[array([[9.99976271e-01, 0.00000000e+00, 1.01697328e-05, 1.35596438e-05],\n [2.33728139e-0...","[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n [2.50816407e-0...","[array([[9.99982840e-01, 0.00000000e+00, 8.57999018e-06, 8.57999018e-06],\n [0.00000000e+0...","[array([[9.99994817e-01, 0.00000000e+00, 1.72754537e-06, 3.45509073e-06],\n [0.00000000e+0...","[array([[9.99994856e-01, 0.00000000e+00, 0.00000000e+00, 5.14449162e-06],\n [2.81811201e-0...","[array([[9.99948807e-01, 0.00000000e+00, 1.15597200e-05, 3.96333257e-05],\n [7.36713374e-0...",1.576453,0.367281
3,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference,1,0.824332,0.159098,0.141585,0.805546,"[array([[9.99957056e-01, 0.00000000e+00, 0.00000000e+00, 4.29439071e-05],\n [6.64359862e-0...","[array([[9.90477220e-01, 2.98110908e-05, 6.99815358e-03, 2.49481566e-03],\n [2.20981045e-0...","[array([[9.99971186e-01, 1.69495547e-06, 0.00000000e+00, 2.71192876e-05],\n [5.84320348e-0...","[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n [2.00653126e-0...","[array([[9.99927070e-01, 0.00000000e+00, 8.57999018e-06, 6.43499263e-05],\n [1.05744015e-0...","[array([[9.99975814e-01, 0.00000000e+00, 5.18263610e-06, 1.90029990e-05],\n [0.00000000e+0...","[array([[9.99987996e-01, 0.00000000e+00, 0.00000000e+00, 1.20038138e-05],\n [1.40905600e-0...","[array([[9.99861283e-01, 0.00000000e+00, 6.27527657e-05, 7.59638742e-05],\n [0.00000000e+0...",1.629878,0.300683
4,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference_18900_10112055,1,0.892019,0.094439,0.250173,0.696974,"[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n [4.42906574e-0...","[array([[9.92319918e-01, 3.72638636e-06, 5.26911031e-03, 2.40724559e-03],\n [3.86716829e-0...","[array([[9.99994915e-01, 

In [71]:
cols = eval_df.columns.tolist()

In [72]:
cols

['model',
 'inf_dir',
 'test_im',
 'test_11',
 'test_12',
 'test_21',
 'test_22',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 'test_11+22',
 'test_12+21']

In [74]:
cols_ = cols[0:7]+cols[-2:]+cols[7:-2]

In [75]:
cols_

['model',
 'inf_dir',
 'test_im',
 'test_11',
 'test_12',
 'test_21',
 'test_22',
 'test_11+22',
 'test_12+21',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8']

In [76]:
eval_df_ = eval_df[cols_]

In [77]:
eval_df_

,model,inf_dir,test_im,test_11,test_12,test_21,test_22,test_11+22,test_12+21,1,2,3,4,5,6,7,8
0,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference_17000_10112050,1,0.918915,0.067399,0.295044,0.649303,1.568218,0.362443,"[array([[9.99997955e-01, 0.00000000e+00, 0.00000000e+00, 2.04494796e-06],\n [6.64359862e-0...","[array([[9.92886328e-01, 2.42215113e-05, 2.34389702e-03, 4.74555302e-03],\n [1.10490523e-0...","[array([[9.99996610e-01, 0.00000000e+00, 0.00000000e+00, 3.38991095e-06],\n [0.00000000e+0...","[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n [0.00000000e+0...","[array([[9.99939940e-01, 0.00000000e+00, 2.14499754e-06, 5.79149337e-05],\n [0.00000000e+0...","[array([[9.99965449e-01, 0.00000000e+00, 0.00000000e+00, 3.45509073e-05],\n [0.00000000e+0...","[array([[9.99974278e-01, 0.00000000e+00, 0.00000000e+00, 2.57224581e-05],\n [0.00000000e+0...","[array([[9.99826604e-01, 0.00000000e+00, 8.25694285e-06, 1.65138857e-04],\n [0.00000000e+0..."
1,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference,1,0.895175,0.097146,0.224378,0.753510,1.648685,0.321524,"[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n [6.64359862e-0...","[array([[9.92843475e-01, 0.00000000e+00, 6.76898081e-03, 3.87544181e-04],\n [1.21539575e-0...","[array([[9.99969491e-01, 0.00000000e+00, 1.01697328e-05, 2.03394657e-05],\n [1.98668918e-0...","[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n [3.00979689e-0...","[array([[9.99974260e-01, 0.00000000e+00, 4.28999509e-06, 2.14499754e-05],\n [0.00000000e+0...","[array([[9.99986180e-01, 0.00000000e+00, 1.72754537e-06, 1.20928176e-05],\n [0.00000000e+0...","[array([[9.99993141e-01, 0.00000000e+00, 0.00000000e+00, 6.85932216e-06],\n [7.04528001e-0...","[array([[9.99922385e-01, 0.00000000e+00, 2.80736057e-05, 4.95416571e-05],\n [0.00000000e+0..."
2,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference_13200_10112050,1,0.926572,0.058502,0.308779,0.649881,1.576453,0.367281,"[array([[9.99991820e-01, 0.00000000e+00, 0.00000000e+00, 8.17979182e-06],\n [1.10726644e-0...","[array([[9.95848806e-01, 0.00000000e+00, 3.22332420e-03, 9.27870203e-04],\n [7.18188397e-0...","[array([[9.99976271e-01, 0.00000000e+00, 1.01697328e-05, 1.35596438e-05],\n [2.33728139e-0...","[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n [2.50816407e-0...","[array([[9.99982840e-01, 0.00000000e+00, 8.57999018e-06, 8.57999018e-06],\n [0.00000000e+0...","[array([[9.99994817e-01, 0.00000000e+00, 1.72754537e-06, 3.45509073e-06],\n [0.00000000e+0...","[array([[9.99994856e-01, 0.00000000e+00, 0.00000000e+00, 5.14449162e-06],\n [2.81811201e-0...","[array([[9.99948807e-01, 0.00000000e+00, 1.15597200e-05, 3.96333257e-05],\n [7.36713374e-0..."
3,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference,1,0.824332,0.159098,0.141585,0.805546,1.629878,0.300683,"[array([[9.99957056e-01, 0.00000000e+00, 0.00000000e+00, 4.29439071e-05],\n [6.64359862e-0...","[array([[9.90477220e-01, 2.98110908e-05, 6.99815358e-03, 2.49481566e-03],\n [2.20981045e-0...","[array([[9.99971186e-01, 1.69495547e-06, 0.00000000e+00, 2.71192876e-05],\n [5.84320348e-0...","[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n [2.00653126e-0...","[array([[9.99927070e-01, 0.00000000e+00, 8.57999018e-06, 6.43499263e-05],\n [1.05744015e-0...","[array([[9.99975814e-01, 0.00000000e+00, 5.18263610e-06, 1.90029990e-05],\n [0.00000000e+0...","[array([[9.99987996e-01, 0.00000000e+00, 0.00000000e+00, 1.20038138e-05],\n [1.40905600e-0...","[array([[9.99861283e-01, 0.00000000e+00, 6.27527657e-05, 7.59638742e-05],\n [0.00000000e+0..."
4,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference_18900_10112055,1,0.892019,0.094439,0.250173,0.696974,1.588993,0.344611,"[array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],\n [4.42906574e-0...","[array([[9.92319918e-01, 3.72638636e-06, 5.26911031e-03, 2.40724559e-03],\n [3.86716829e-0...","[array(

In [78]:
eval_df_.to_csv('/proj/NIRAL/users/siyangj/inference_eval_2.csv',index=None)

In [79]:
pd.options.display.max_colwidth = 100

In [82]:
eval_df_.sort_values('test_11+22',ascending=False).iloc[:,0:9]

,model,inf_dir,test_im,test_11,test_12,test_21,test_22,test_11+22,test_12+21
5,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference_21900_10112055,1,0.861016,0.125160,0.120896,0.821308,1.682324,0.246056
1,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference,1,0.895175,0.097146,0.224378,0.753510,1.648685,0.321524
7,10112056,/proj/NIRAL/users/siyangj/model_10112056/inference_16500_10112056,1,0.793855,0.189398,0.090426,0.849576,1.643431,0.279823
8,10112056,/proj/NIRAL/users/siyangj/model_10112056/inference,1,0.845575,0.147383,0.184313,0.789043,1.634618,0.331696
3,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference,1,0.824332,0.159098,0.141585,0.805546,1.629878,0.300683
10,10112103,/proj/NIRAL/users/siyangj/model_10112103/inference_12300_10112103,1,0.809434,0.176941,0.130890,0.809682,1.619116,0.307831
9,10112103,/proj/NIRAL/users/siyangj/model_10112103/inference,1,0.721373,0.249500,0.052741,0.869897,1.591270,0.302241
4,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference_18900_10112055,1,0.892019,0.094439,0.250173,0.696974,1.588993,0.344611
12,10112127,/proj/NIRAL/users/siyangj/model_10112127/inference_8700_10112127,1,0.713201,0.271862,0.050931,0.864683,1.577885,0.322792
2,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference_13200_10112050,1,0.926572,0.058502,0.308779,0.649881,1.576453,0.367281


In [83]:
eval_df_.sort_values('test_12+21',ascending=True).iloc[:,0:9]

,model,inf_dir,test_im,test_11,test_12,test_21,test_22,test_11+22,test_12+21
5,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference_21900_10112055,1,0.861016,0.125160,0.120896,0.821308,1.682324,0.246056
7,10112056,/proj/NIRAL/users/siyangj/model_10112056/inference_16500_10112056,1,0.793855,0.189398,0.090426,0.849576,1.643431,0.279823
6,10112056,/proj/NIRAL/users/siyangj/model_10112056/inference_18100_10112056,1,0.825539,0.124662,0.174211,0.737446,1.562985,0.298872
3,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference,1,0.824332,0.159098,0.141585,0.805546,1.629878,0.300683
13,10112130,/proj/NIRAL/users/siyangj/model_10112130/inference_7300,1,0.712039,0.238079,0.062735,0.804453,1.516491,0.300814
9,10112103,/proj/NIRAL/users/siyangj/model_10112103/inference,1,0.721373,0.249500,0.052741,0.869897,1.591270,0.302241
11,10112127,/proj/NIRAL/users/siyangj/model_10112127/inference_13500_10112127,1,0.737849,0.235875,0.069771,0.803042,1.540891,0.305646
10,10112103,/proj/NIRAL/users/siyangj/model_10112103/inference_12300_10112103,1,0.809434,0.176941,0.130890,0.809682,1.619116,0.307831
1,10112050,/proj/NIRAL/users/siyangj/model_10112050/inference,1,0.895175,0.097146,0.224378,0.753510,1.648685,0.321524
12,10112127,/proj/NIRAL/users/siyangj/model_10112127/inference_8700_10112127,1,0.713201,0.271862,0.050931,0.864683,1.577885,0.322792
